## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [5]:
import numpy as np
%load_ext line_profiler
A = np.random.randint(0, 1000, size = (1_000_000))

In [ ]:
pip install line_profiler
%load_ext line_profiler
%lprun -f f1 f1(A)

In [6]:
def f1(A):
    acc,cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt +=1
    return acc/cnt

In [7]:
%timeit f1(A)

425 ms ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%lprun -f f1 f1(A)

In [9]:
def f2(A):
    acc = 0
    for ai in A:
        bi = ai + 100
        acc += bi
    return acc/ len(A)

In [16]:
%timeit f2(A)

484 ms ± 84.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%lprun -f f2 f2(A)

In [20]:
def f3(A):
    acc = 0
    for ai in A:
        acc += ai
    return acc/ len(A) + 100

In [21]:
%timeit f3(A)

150 ms ± 62.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%lprun -f f3 f3(A)

In [23]:
def f4(A):
    return A.mean() + 100

In [24]:
%timeit f4(A)

1.39 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%lprun -f f4 f4(A)

In [26]:
import numba

In [28]:
@numba.njit
def f5(A):
    acc,cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt +=1
    return acc/cnt

In [30]:
%timeit f5(A)

189 µs ± 33.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [1]:
%lprun -f f5 f5(A)

UsageError: Line magic function `%lprun` not found.


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [32]:
import pandas  as pd
df = pd.DataFrame(np.random.randint(0,1000,size=(2_000_000, 4)), columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)
df.head

<bound method NDFrame.head of          col1  col2  col3  col4 key
0         410   755   245    57   f
1         580   417    83    67   h
2         537   317   590   956   f
3         112   359   910   217   g
4          81   601   480   753   a
5         397    94   637   409   a
6         761   848   375   109   g
7         178   302   747   497   g
8         230   653   440   534   b
9         403   274   252   442   g
10        961    59   594   386   h
11        377   274   645   998   g
12        917   375   249   914   b
13        754   912   460   290   b
14        321   853   665   498   a
15        463   682   494   950   c
16        275    61   715   658   c
17        417   375   815   740   h
18        497   630   265   325   a
19        347   602   229   219   b
20         14   390   991   350   e
21        584   205   884   156   b
22        412   299   591   611   e
23        303   674   773   612   e
24        283   506   846   149   e
25         76   136   408   294   

In [33]:
def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis = 0)

In [34]:
%timeit g(df)

1.02 s ± 63.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Лабораторная работа 3

In [3]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [14]:
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv',sep = ',', header=0,parse_dates=['submitted'])
recipes['submitted'] = pd.to_datetime(recipes['submitted'], format='%Y-%m-%d')
reviews = pd.read_csv('reviews_sample.csv',sep = ',', header=0, index_col = 0)
reviews['date'] = pd.to_datetime(reviews['date'], format='%Y-%m-%d')
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [9]:
def f(reviews):
    cnt = 0
    for index, row in reviews.iterrows():
        cnt+=row["rating"]
    return cnt/len(reviews['rating'])
f(reviews)

4.410802235271832

def f(reviews):
    cnt = 0
    cnt1 = 0
    for index, row in reviews.iterrows():
        if(row['date'].year == 2010)
            cnt+=row["rating"]
            cnt1+=1
    return cnt/cnt1
f(reviews)

In [10]:
%timeit f(reviews)

9.62 s ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
a = reviews.loc[:,'date':'rating']
def f1(a):
    cnt = 0
    for index, row in a.iterrows():
        cnt+=row['rating']
    return cnt/len(a['rating'])
f(a)

4.410802235271832

In [20]:
%timeit f1(a)

10.6 s ± 314 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
def f2(reviews):
    return reviews['rating'].mean()
f(reviews)

4.410802235271832

In [22]:
%timeit f2(reviews)

226 µs ± 26.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [23]:
%lprun -f f f(reviews)

In [25]:
#Timer unit: 1e-07 s

#Total time: 33.274 s
#File: <ipython-input-18-4bd3fce3f39c>
#Function: f at line 2
#
#Line #      Hits         Time  Per Hit   % Time  Line Contents
#==============================================================
#     2                                           def f(a):
#     3         1        140.0    140.0      0.0      cnt = 0
#     4    126697  296140723.0   2337.4     89.0      for index, row in a.iterrows():
#     5    126696   36599264.0    288.9     11.0          cnt+=row['rating']
#     6         1        221.0    221.0      0.0      return cnt/len(a['rating'])

In [24]:
%lprun -f f1 f1(a)

In [ ]:
#Timer unit: 1e-07 s

#Total time: 44.3183 s
#File: <ipython-input-19-e1cac7b8b192>
#Function: f1 at line 2
#
#Line #      Hits         Time  Per Hit   % Time  Line Contents
#==============================================================
#     2                                           def f1(a):
#     3         1         88.0     88.0      0.0      cnt = 0
#     4    126697  393972209.0   3109.6     88.9      for index, row in a.iterrows():
#     5    126696   49210031.0    388.4     11.1          cnt+=row['rating']
#     6         1        319.0    319.0      0.0      return cnt/len(a['rating'])

In [28]:
%lprun -f f2 f2(reviews)

In [ ]:
#Timer unit: 1e-07 s
#
#Total time: 0.001829 s
#File: <ipython-input-21-8814a385e6b5>
#Function: f2 at line 1
#
#Line #      Hits         Time  Per Hit   % Time  Line Contents
#==============================================================
#     1                                           def f2(reviews):
#     2         1      18290.0  18290.0    100.0      return reviews['rating'].mean()

In [30]:
a = reviews.loc[:,'date':'rating']
def f3(a):
    a = list(a['rating'])
    cnt = 0
    for i in a:
        cnt+=i
    return cnt/len(a)
f3(a)

4.410802235271832

In [31]:
%timeit f3(a)

23.9 ms ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
import numba
import numpy as np
a = np.array(reviews['rating'])
@numba.njit
def f4(x):
    cnt = 0
    for i in range(len(x)):
        cnt+=x[i]
    return cnt/len(x)
f4(a)

4.410802235271832

In [9]:
%timeit f4(a)

44.5 µs ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [16]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [17]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

In [ ]:
#Timer unit: 1e-07 s
#
#Total time: 96.3174 s
#File: <ipython-input-67-b1bc049bcd0c>
#Function: get_word_reviews_count at line 1
#
#Line #      Hits         Time  Per Hit   % Time  Line Contents
#==============================================================
#     1                                           def get_word_reviews_count(df):
#     2         1        101.0    101.0      0.0      word_reviews = {}
#     3    126680  296279981.0   2338.8     30.8      for _, row in df.dropna(subset=['review']).iterrows():
#     4    126679   63189065.0    498.8      6.6          recipe_id, review = row['recipe_id'], row['review']
#     5    126679    7155451.0     56.5      0.7          words = review.split(' ')
#     6   6918689   32287048.0      4.7      3.4          for word in words:
#     7   6792010   41860255.0      6.2      4.3              if word not in word_reviews:
#     8    174944    1362566.0      7.8      0.1                  word_reviews[word] = []
#     9   6792010   46837912.0      6.9      4.9              word_reviews[word].append(recipe_id)
#    10                                               
#    11         1          8.0      8.0      0.0      word_reviews_count = {}
#    12    126680  320519413.0   2530.2     33.3      for _, row in df.dropna(subset=['review']).iterrows():
#    13    126679   40115215.0    316.7      4.2          review = row['review']
#    14    126679    7930762.0     62.6      0.8          words = review.split(' ')
#    15   6918689   36827930.0      5.3      3.8          for word in words:
#    16   6792010   68807951.0     10.1      7.1              word_reviews_count[word] = len(word_reviews[word])
#    17         1          8.0      8.0      0.0      return word_reviews_count

In [54]:
def get_word_reviews_count1(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = 0
            word_reviews[word] +=1
    return word_reviews

In [56]:
%lprun -f get_word_reviews_count1 get_word_reviews_count1(reviews)

In [ ]:
#Timer unit: 1e-07 s
#
#Total time: 40.1797 s
#File: <ipython-input-54-60a7f21a52da>
#Function: get_word_reviews_count1 at line 1
#
#Line #      Hits         Time  Per Hit   % Time  Line Contents
#==============================================================
#     1                                           def get_word_reviews_count1(df):
#     2         1        143.0    143.0      0.0      word_reviews = {}
#     3         1          8.0      8.0      0.0      word_reviews_count = {}
#     4    126680  242783767.0   1916.5     60.4      for _, row in df.dropna(subset=['review']).iterrows():
#     5    126679   52134830.0    411.6     13.0          recipe_id, review = row['recipe_id'], row['review']
#     6    126679    5952544.0     47.0      1.5          words = review.split(' ')
#     7   6918689   26264105.0      3.8      6.5          for word in words:
#     8   6792010   36156284.0      5.3      9.0              if word not in word_reviews:
#     9    174944     952173.0      5.4      0.2                  word_reviews[word] = 0
#    10   6792010   37553389.0      5.5      9.3              word_reviews[word] +=1
#    11         1          8.0      8.0      0.0      return word_reviews

In [11]:
def get_word_reviews_count2(reviews):
    reviews = reviews.dropna(subset=['review'])
    list_str = list(reviews['review'].str.split(' '))
    words = []
    word_reviews = {}
    for i in list_str:
        words+=i
    for word in words:
        if word not in word_reviews:
            word_reviews[word] = 0
        word_reviews[word] += 1
    return(word_reviews)

In [ ]:
#Timer unit: 1e-07 s
#
#Total time: 10.2779 s
#File: <ipython-input-59-40c9e59efba6>
#Function: get_word_reviews_count2 at line 1
#
#Line #      Hits         Time  Per Hit   % Time  Line Contents
#==============================================================
#     1                                           def get_word_reviews_count2(reviews):
#     2         1     358275.0 358275.0      0.3      reviews = reviews.dropna(subset=['review'])
#     3         1    9905948.0 9905948.0      9.6      list_str = list(reviews['review'].str.split(' '))
#     4         1         14.0     14.0      0.0      words = []
#     5         1          7.0      7.0      0.0      word_reviews = {}
#     6    126680     467909.0      3.7      0.5      for i in list_str:
#     7    126679    3119653.0     24.6      3.0          words+=i
#     8   6792011   23933001.0      3.5     23.3      for word in words:
#     9   6792010   30380574.0      4.5     29.6          if word not in word_reviews:
#    10    174944     807053.0      4.6      0.8              word_reviews[word] = 0
#    11   6792010   33806637.0      5.0     32.9          word_reviews[word] += 1
#    12         1          5.0      5.0      0.0      return(word_reviews)

In [18]:
def get_word_reviews_count3(reviews):
    reviews = reviews.dropna(subset=['review'])
    list_str = list(reviews['review'].str.split(' '))
    words = []
    word_reviews = {}
    for i in list_str:
        words+=i
    words_list = list(set(words))
    for word in words_list:
        word_reviews[word] = 0
    for word in words:
        word_reviews[word] +=1
    return(word_reviews)

In [19]:
%lprun -f get_word_reviews_count3 get_word_reviews_count3(reviews)

In [ ]:
#Timer unit: 1e-07 s
#
#Total time: 7.73803 s
#File: <ipython-input-69-4f3709db9b12>
#Function: get_word_reviews_count3 at line 1
#
#Line #      Hits         Time  Per Hit   % Time  Line Contents
#==============================================================
#     1                                           def get_word_reviews_count3(reviews):
#     2         1     382837.0 382837.0      0.5      reviews = reviews.dropna(subset=['review'])
#     3         1    8401546.0 8401546.0     10.9      list_str = list(reviews['review'].str.split(' '))
#     4         1         12.0     12.0      0.0      words = []
#     5         1          5.0      5.0      0.0      word_reviews = {}
#     6    126680     474594.0      3.7      0.6      for i in list_str:
#     7    126679    2637419.0     20.8      3.4          words+=i
#     8         1    4917956.0 4917956.0      6.4      words_list = list(set(words))
#     9    174945     761675.0      4.4      1.0      for word in words_list:
#    10    174944     821596.0      4.7      1.1          word_reviews[word] = 0
#    11   6792011   23258709.0      3.4     30.1      for word in words:
#    12   6792010   35723994.0      5.3     46.2          word_reviews[word] +=1
#    13         1          6.0      6.0      0.0      return(word_reviews)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва. 
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [25]:
reviews = reviews.drop(reviews.index[(reviews.rating.eq(0))])

In [51]:
def MAPE_A(reviews):
    result = []
    mn = reviews['rating'].mean()
    for i in reviews['rating']:
        result.append(abs((i-mn)/i)*100)
    return np.mean(result)

In [53]:
MAPE_A(reviews)
%timeit MAPE_A(reviews)

37.7 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
import numpy as np
rev = np.array(reviews['rating'])
@numba.njit
def MAPE_B(rev1):
    result = 0
    mn = np.mean(rev)
    for i in rev1:
            result+= (abs((i-mn)/i)*100)
    return result/len(rev1)

In [27]:
MAPE_B(rev)

16.266663338761987

In [94]:
rev = np.array(reviews['rating'])
def MAPE_C(rev1):
    result = 0
    mn = np.mean(rev)
    for i in rev1:
            result+= (abs((i-mn)/i)*100)
    return result/len(rev1)

In [95]:
MAPE_C(rev)
%timeit MAPE_C(rev)

357 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
rev = np.array(reviews['rating'])
@numba.njit
def MAPE_D(rev1):
    result = 0
    mn = np.mean(rev)
    for i in rev1:
            result+= (abs((i-mn)/i)*100)
    return result/len(rev1)

In [93]:
MAPE_D(rev)
%timeit MAPE_D(rev)

277 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
